The following is an excerpt from Chapter 5 of Watson's Data Management 6th Edition

> Consider the case when items are sold. We can immediately identify two entities: SALE and ITEM. A sale can contain many items, and an item can appear in many sales. We are not saying the same item can be sold many times, but the particular type of item (e.g., a compass) can be sold many times; thus we have a many-to-many (m:m) relationship between SALE and ITEM. When we have an m:m relationship, we create a third entity to link the entities through two 1:m relationships. Usually, it is fairly easy to name this third entity. In this case, this third entity, typically known as an associative entity, is called LINE ITEM. A typical sales form lists the items purchased by a customer. Each of the lines appearing on the order form is generally known in retailing as a line item, which links an item and a sale.

The following ERD shows the data schema

![ERD](images/sale_erd.png)

The file `data/sales_data.xlsx` is an Excel spreadsheet containing three workbooks with data about `sale`, `item`, and `lineitem`. The code below loads the data into Pandas data frames. Use the data frames to answer the following questions:

- List the name, quantity, price, and value of items sold on January 16, 2021
- List items that were sold on January 16, 2021, or are brown
- List items that were sold on January 16, 2021, and are brow
- Report all clothing items (type “C”) for which a sale is recorded
- Report all clothing items that have not been sold
- Find the items that have appeared in all sales


In [51]:
import pandas as pd

In [52]:
sale = pd.read_excel('data/sales_data.xlsx', sheet_name='sale')
item = pd.read_excel('data/sales_data.xlsx', sheet_name='item') 
lineitem = pd.read_excel('data/sales_data.xlsx', sheet_name='lineitem') 

In [53]:
data = pd.merge( pd.merge(sale, lineitem), item)
data['TOTAL'] = data.LINEQTY * data.LINEPRICE
data.head()

,SALENO,SALEDATE,SALETEXT,LINENO,LINEQTY,LINEPRICE,ITEMNO,ITEMNAME,ITEMTYPE,ITEMCOLOR,TOTAL
0,1,2021-01-15,NaN,1,1,4.50,2,Pocket knife - Avon,E,Brown,4.50
1,2,2021-01-15,NaN,4,1,2.25,2,Pocket knife - Avon,E,Brown,2.25
2,3,2021-01-15,NaN,2,1,2.25,2,Pocket knife - Avon,E,Brown,2.25
3,4,2021-01-15,NaN,6,1,2.25,2,Pocket knife - Avon,E,Brown,2.25
4,5,2021-01-16,NaN,5,1,0.00,2,Pocket knife - Avon,E,Brown,0.00


In [54]:
jan_16_items = data.loc[data.SALEDATE=='2021-01-16', ['ITEMNAME', 'LINEQTY', 'LINEPRICE', 'TOTAL']]
jan_16_items

,ITEMNAME,LINEQTY,LINEPRICE,TOTAL
4,Pocket knife - Avon,1,0.0,0.0
12,Tent - 2 person,1,60.0,60.0
15,Safari chair,50,36.0,1800.0
16,Hammock,50,40.5,2025.0
17,Tent - 8 person,8,153.0,1224.0


Using set operations

In [59]:
brown_items = item.loc[item.ITEMCOLOR == 'Brown', 'ITEMNAME']

In [60]:
jan_16_items_or_brown = set(jan_16_items.ITEMNAME) | set(brown_items)
jan_16_items_or_brown

{'Hammock',
 'Map case',
 'Pocket knife - Avon',
 'Pocket knife - Nile',
 'Safari chair',
 'Stetson',
 'Tent - 2 person',
 'Tent - 8 person'}

Using joins

In [63]:
pd.merge(jan_16_items, brown_items, how='outer').ITEMNAME

0    Pocket knife - Avon
1        Tent - 2 person
2           Safari chair
3                Hammock
4        Tent - 8 person
5    Pocket knife - Nile
6               Map case
7                Stetson
Name: ITEMNAME, dtype: object

Using set operations

In [64]:
jan_16_items_and_brown = set(jan_16_items.ITEMNAME) & set(brown_items)
jan_16_items_and_brown

{'Pocket knife - Avon'}

Using joins

In [65]:
pd.merge(jan_16_items, brown_items, how='inner').ITEMNAME

0    Pocket knife - Avon
Name: ITEMNAME, dtype: object

In [18]:
sold_C_items = data.loc[data.ITEMTYPE=='C', 'ITEMNAME']
sold_C_items

5     Hat - Polar explorer
6              Pith helmet
7                  Stetson
10     Boots - snake proof
Name: ITEMNAME, dtype: object

In [21]:
sold_and_unsold_items = pd.merge(item, lineitem, how='outer')
sold_and_unsold_items['UNSOLD'] = sold_and_unsold_items.SALENO.isnull()
sold_and_unsold_items.head()

,ITEMNO,ITEMNAME,ITEMTYPE,ITEMCOLOR,LINENO,LINEQTY,LINEPRICE,SALENO,UNSOLD
0,1,Pocket knife - Nile,E,Brown,NaN,NaN,NaN,NaN,True
1,2,Pocket knife - Avon,E,Brown,1.0,1.0,4.50,1.0,False
2,2,Pocket knife - Avon,E,Brown,4.0,1.0,2.25,2.0,False
3,2,Pocket knife - Avon,E,Brown,2.0,1.0,2.25,3.0,False
4,2,Pocket knife - Avon,E,Brown,6.0,1.0,2.25,4.0,False


In [23]:
unsold_C_items = sold_and_unsold_items.loc[sold_and_unsold_items.UNSOLD & (sold_and_unsold_items.ITEMTYPE == 'C'), 'ITEMNAME']
unsold_C_items

11    Hat - Polar explorer
12     Boots - snake proof
20             Pith helmet
25                 Stetson
Name: ITEMNAME, dtype: object

Items on all sales:
- count sales
- count number of sales per item
- select items with equal value

In [24]:
tot_sales = len(sale)


In [42]:
cube = data.set_index(['ITEMNO', 'SALENO']).LINEQTY

In [43]:
sale_count = cube.count(level='ITEMNO')

In [44]:
items_on_all_sales = sale_count[sale_count == tot_sales]

In [46]:
items_on_all_sales

ITEMNO
2    5
Name: LINEQTY, dtype: int64

In [50]:
pd.merge(items_on_all_sales, item, left_index=True, right_on='ITEMNO').ITEMNAME

1    Pocket knife - Avon
Name: ITEMNAME, dtype: object